In [ ]:
import pandas as pd
import numpy as np
import csv
from pprint import pprint
from nanoHUB.application import Application
from nanoHUB.configuration import ClusteringConfiguration
from nanoHUB.pipeline.geddes.data import get_default_s3_client
from nanoHUB.dataaccess.lake import S3FileMapper
from nanoHUB.clustering.infra import create_clusters_repository, get_tool_users, get_user_tools, add_cluster_info, get_clustered_one_day_users, get_unclassified_one_day_users
from ast import literal_eval
import os


pd.set_option('display.max_columns', None)

# logger.debug('Testing')
application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')

#salesforce = application.new_salesforce_engine()

s3_client = get_default_s3_client(application)
raw_mapper = S3FileMapper(s3_client, ClusteringConfiguration().bucket_name_raw)
processed_mapper = S3FileMapper(s3_client, ClusteringConfiguration().bucket_name_processed)

cluster_repo = create_clusters_repository(application, ClusteringConfiguration().bucket_name_processed)

derived_data_file_path = 'derived_data_for_users_with_one_day_users.csv'

In [ ]:
tool_users_df = get_tool_users(processed_mapper)
display(tool_users_df)
len(tool_users_df)
#display(tool_users_df['names_users'])

In [ ]:
#tool_names_query = input("Enter a tool name?")
for index in tool_users_df.index:
    if tool_users_df['toolname'][index] == 'zeno':
        display(tool_users_df.at[index,'names_users'])
        

In [ ]:
#tool_users_df.at[index,'names_users'].to_csv('/nanoHUB/class-cluster/visualization_V1/Dheeraj/tool-ranking-by-user-groups.csv')

In [ ]:
current_dir = os.getcwd()

dict = {"names_of_users":tool_users_df.at[index,'names_users']}
sirs = pd.DataFrame(dict)
sirs.to_csv(current_dir + '/Dheeraj.csv')

# list_rows = tool_users_df.at[index,'names_users']
# np.savetxt("nanoHUB/class-cluster/visualization_V1/Dheeraj.csv", list_rows, delimiter =",",fmt ='% s')



#with open('/nanoHUB/class-cluster/visualization_V1/Dheeraj.csv', 'w', newline=' ') as f:
#    writer = csv.writer(f)
#    writer.writerows(tool_users_df.at[index,'names_users'])

In [ ]:

display(tool_users_df['names_users'])


In [ ]:
for 'toolnames' in tools_users_df:
    if tool_users_df.at[2,'toolname'] == '1dfdmht':
        display(tool_users_df.at[117,'names_users'])

In [ ]:
display(tool_users_df.head())

In [ ]:
if tool_users_df.at[2,'toolname'] == '1dfdmht':
    display(tool_users_df.at[2,'names_users'])
    
    

In [ ]:
display(tool_users_df.at[117,'names_users'])

In [ ]:
classified_one_day_users_df = get_clustered_one_day_users(processed_mapper)

display(len(classified_one_day_users_df))
display(classified_one_day_users_df.head())
display(classified_one_day_users_df.tail())

In [ ]:
unclassified_one_day_users_df = get_unclassified_one_day_users(processed_mapper)

display(len(unclassified_one_day_users_df))
display(unclassified_one_day_users_df.head())
display(unclassified_one_day_users_df.tail())

In [ ]:
unclassified_user_tools_df = tool_users_df.loc[tool_users_df['user'].isin(unclassified_one_day_users_df['username']),:]

display(len(unclassified_user_tools_df))
display(unclassified_user_tools_df.head())
display(unclassified_user_tools_df.tail())

In [ ]:
a = pd.Series([item for sublist in unclassified_user_tools_df['names_tools'] for item in sublist])
df = a.groupby(a).size().rename_axis('names_tools').reset_index(name='frequency')


display(df.head(10))

In [ ]:
sorted_tool_count_for_unclassfied_users = df.sort_values('frequency', ascending=False)
sorted_tool_count_for_unclassfied_users = sorted_tool_count_for_unclassfied_users.set_index('names_tools').reset_index()
display(sorted_tool_count_for_unclassfied_users)

sorted_tool_count_for_unclassfied_users.to_csv('sorted_tool_count_for_unclassified_one_day_users.csv', index=False)

In [ ]:
classified_user_tools_df = tool_users_df.loc[tool_users_df['user'].isin(classified_one_day_users_df['username']),:]

display(len(classified_user_tools_df))
display(classified_user_tools_df.head())
display(classified_user_tools_df.tail())

In [ ]:
a = pd.Series([item for sublist in classified_user_tools_df['names_tools'] for item in sublist])
df = a.groupby(a).size().rename_axis('names_tools').reset_index(name='frequency')


display(df.head(10))

In [ ]:
sorted_tool_count_for_clustered_users = df.sort_values('frequency', ascending=False)
sorted_tool_count_for_clustered_users = sorted_tool_count_for_clustered_users.set_index('names_tools').reset_index()
display(sorted_tool_count_for_clustered_users)

sorted_tool_count_for_clustered_users.to_csv('sorted_tool_count_for_clustered_one_day_users.csv', index=False)

In [ ]:
sorted_tools_unclassified_users = sorted_tool_count_for_unclassfied_users['names_tools'].to_list()
sorted_tools_clustered_users = sorted_tool_count_for_clustered_users['names_tools'].to_list()

In [ ]:
import difflib
sm = difflib.SequenceMatcher(None, sorted_tools_unclassified_users, sorted_tools_clustered_users)
similarity_ratio = sm.ratio()

display(similarity_ratio)

In [ ]:
common_tools = list(set(sorted_tools_unclassified_users) & set(sorted_tools_clustered_users))

display(len(common_tools))
# display(common_tools)

In [ ]:
total_tools_count = len(sorted_tools_clustered_users + sorted_tools_unclassified_users)
common_tools_count = len(common_tools)

percentage_common_tools = (common_tools_count*100)/total_tools_count

display(percentage_common_tools)